In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from numpy import random
from scipy import stats
from sklearn.model_selection import StratifiedKFold

#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")

In [2]:
dataset_red = pd.read_csv('DB/data-selected.txt',  engine='python')

#Se carga la base de datos 
X = dataset_red.iloc[:,1:54]
Y = dataset_red.iloc[:,54]

In [3]:
def MAPE(Y_est,Y):
    N = np.size(Y)
    mape = np.sum(abs((Y_est.reshape(N,1) - Y.reshape(N,1))/Y.reshape(N,1)))/N
    return mape

In [8]:
#https://github.com/jmetzen/kernel_regression
"""The :mod:`sklearn.kernel_regressor` module implements the Kernel Regressor.
"""
# Author: Jan Hendrik Metzen <janmetzen@mailbox.de>
#
# License: BSD 3 clause

import numpy as np

from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.base import BaseEstimator, RegressorMixin


class KernelRegression(BaseEstimator, RegressorMixin):
    """Nadaraya-Watson kernel regression with automatic bandwidth selection.

    This implements Nadaraya-Watson kernel regression with (optional) automatic
    bandwith selection of the kernel via leave-one-out cross-validation. Kernel
    regression is a simple non-parametric kernelized technique for learning
    a non-linear relationship between input variable(s) and a target variable.

    Parameters
    ----------
    kernel : string or callable, default="rbf"
        Kernel map to be approximated. A callable should accept two arguments
        and the keyword arguments passed to this object as kernel_params, and
        should return a floating point number.

    gamma : float, default=None
        Gamma parameter for the RBF ("bandwidth"), polynomial,
        exponential chi2 and sigmoid kernels. Interpretation of the default
        value is left to the kernel; see the documentation for
        sklearn.metrics.pairwise. Ignored by other kernels. If a sequence of
        values is given, one of these values is selected which minimizes
        the mean-squared-error of leave-one-out cross-validation.

    See also
    --------

    sklearn.metrics.pairwise.kernel_metrics : List of built-in kernels.
    """

    def __init__(self, kernel="rbf", gamma=None):
        self.kernel = kernel
        self.gamma = gamma

    def fit(self, X, y):
        """Fit the model

        Parameters
        ----------
        X : array-like of shape = [n_samples, n_features]
            The training input samples.

        y : array-like, shape = [n_samples]
            The target values

        Returns
        -------
        self : object
            Returns self.
        """
        self.X = X
        self.y = y

        if hasattr(self.gamma, "__iter__"):
            self.gamma = self._optimize_gamma(self.gamma)

        return self

    def predict(self, X):
        """Predict target values for X.

        Parameters
        ----------
        X : array-like of shape = [n_samples, n_features]
            The input samples.

        Returns
        -------
        y : array of shape = [n_samples]
            The predicted target value.
        """
        K = pairwise_kernels(self.X, X, metric=self.kernel, gamma=self.gamma)
        return (K * self.y[:, None]).sum(axis=0) / K.sum(axis=0)

    def _optimize_gamma(self, gamma_values):
        # Select specific value of gamma from the range of given gamma_values
        # by minimizing mean-squared error in leave-one-out cross validation
        mse = np.empty_like(gamma_values, dtype=np.float)
        for i, gamma in enumerate(gamma_values):
            K = pairwise_kernels(self.X, self.X, metric=self.kernel,
                                 gamma=gamma)
            np.fill_diagonal(K, 0)  # leave-one-out
            Ky = K * self.y[:, np.newaxis]
            y_pred = Ky.sum(axis=0) / K.sum(axis=0)
            mse[i] = ((y_pred - self.y) ** 2).mean()

        return gamma_values[np.nanargmin(mse)]


In [9]:
#Mean Absolute Percentage Error para los problemas de regresión
def MAPE(Y_est,Y):
    N = np.size(Y)
    mape = np.sum(abs((Y_est.reshape(N,1) - Y.reshape(N,1))/Y.reshape(N,1)))/N
    return mape 

In [10]:
print(__doc__)

import time
from sklearn.model_selection import KFold
from sklearn import preprocessing
#https://jakevdp.github.io/PythonDataScienceHandbook/05.13-kernel-density-estimation.html
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score


Folds = 2
Errores = np.ones(Folds)
Scores = np.ones(Folds)
j = 0
sum = 0
kf = KFold(n_splits=Folds)

kr = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

for train_index, test_index in kf.split(X):
        
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]  
       
    
    
    #Normalizamos los datos
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    t0 = time.time()
    
    ykr = kr.fit(X_train, y_train)
    y_pred = kr.predict(X_train)
    y_pred_val= kr.predict(X_test)
    print("KR including bandwith fitted in %.3f s" + str((time.time() - t0)))
    
    #kde = KernelDensity(bandwidth=1.0, kernel='gaussian')
    #kde.fit(X_train)
    # instantiate and fit the KDE model
    #bandwidths = 10 ** np.linspace(-1, 1, 100)
    #grid = GridSearchCV(KernelDensity(kernel='gaussian'),{'bandwidth': bandwidths},cv=LeaveOneOut(len(X_train)))
    #grid.fit(X_train);

print("ECM entrenamiento: "+ str(MAPE(np.array(y_pred),np.array(y_train))) 
print("ECM validacion: "+ str(MAPE(y_pred_val,y_test)))

The :mod:`sklearn.kernel_regressor` module implements the Kernel Regressor.



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in true_divide


KR including bandwith fitted in %.3f s25.469080209732056
KR including bandwith fitted in %.3f s9.027398824691772


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in true_divide


AttributeError: 'Series' object has no attribute 'reshape'

In [21]:
y_train[1]

0

In [23]:
ind = y_train != 0
MAPE(np.array(y_pred[ind]),np.array(y_train[ind]))

0.002943471157719599

In [24]:
print(np.shape(y_train))

(5000,)
